# 5.5 TensorFlow最佳实践样例程序

In [1]:
purpose = 'train'
purpose = 'eval'

In [2]:
import tensorflow as tf

INPUT_NODE = 784
OUTPUT_NODE = 10
LAYER1_NODE = 500

def get_weight_variable(shape, regularizer):
    weights = tf.get_variable("weights", shape,
                             initializer=tf.truncated_normal_initializer(stddev=0.1))
    if regularizer != None:
        tf.add_to_collection('losses', regularizer(weights))
    return weights

def inference(input_tensor, regularizer):
    with tf.variable_scope('layer1'):
        weights = get_weight_variable([INPUT_NODE, LAYER1_NODE], regularizer)
        biases = tf.get_variable('biases', [LAYER1_NODE], initializer=tf.constant_initializer(0.0))
        layer1 = tf.nn.relu(tf.matmul(input_tensor, weights) + biases)
        
    with tf.variable_scope('layer2'):
        weights = get_weight_variable([LAYER1_NODE, OUTPUT_NODE], regularizer)
        biases = tf.get_variable('biases', [OUTPUT_NODE], initializer=tf.constant_initializer(0.0))
        layer2 = tf.matmul(layer1, weights) + biases
        
    return layer2

In [3]:
import os
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

BATCH_SIZE = 100
LEARNING_RATE_BASE = 0.8
LEARNING_RATE_DECAY = 0.99
REGULARIZATION_RATE = 0.0001
TRAINING_STEPS = 30000
MOVING_AVERAGE_DECAY = 0.99
MODEL_SAVE_PATH = 'model'
MODEL_NAME = 'model_55.ckpt'

def train(mnist):
    x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
    y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
    
    regularizer = tf.contrib.layers.l2_regularizer(REGULARIZATION_RATE)
    y = inference(x, regularizer)
    
    global_step = tf.Variable(0, trainable=False)
    
    variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY, global_step)
    variables_average_op = variable_averages.apply(tf.trainable_variables())
    cross_entropy = tf.nn.sparse_softmax_cross_entropy_with_logits(logits=y, labels=tf.argmax(y_, 1))
    cross_entropy_mean = tf.reduce_mean(cross_entropy)
    loss = cross_entropy_mean + tf.add_n(tf.get_collection('losses'))
    learning_rate = tf.train.exponential_decay(LEARNING_RATE_BASE, global_step, mnist.train.num_examples / BATCH_SIZE,
                                              LEARNING_RATE_DECAY)
    train_step = tf.train.GradientDescentOptimizer(learning_rate).minimize(loss, global_step=global_step)
    with tf.control_dependencies([train_step, variables_average_op]):
        train_op = tf.no_op(name='train')
        
    saver = tf.train.Saver()
    with tf.Session() as sess:
        tf.global_variables_initializer().run()
        
        for i in range(TRAINING_STEPS):
            xs, ys = mnist.train.next_batch(BATCH_SIZE)
            _, loss_value, step = sess.run([train_op, loss, global_step], feed_dict={x: xs, y_: ys})
            
            if i % 1000 == 999:
                print("After %d training steps, loss on training batch is %g." % (step, loss_value))
                saver.save(sess, os.path.join(MODEL_SAVE_PATH, MODEL_NAME), global_step=global_step)

if purpose == 'train':
    if os.path.exists("/Users"):
        mnist = input_data.read_data_sets("/Users/colinzuo/work/github/personal_study/"
                                          "tensorflow/book_caicloud/mnist/", one_hot=True)
    else:
        mnist = input_data.read_data_sets("D:\\work\\DataAnalysis\\study\\mnist", one_hot=True)
        
    train(mnist)

In [ ]:
import time
import tensorflow as tf
from tensorflow.examples.tutorials.mnist import input_data

EVAL_INTERVAL_SECS = 10

def evaluate(mnist):
    with tf.Graph().as_default() as g:
        x = tf.placeholder(tf.float32, [None, INPUT_NODE], name='x-input')
        y_ = tf.placeholder(tf.float32, [None, OUTPUT_NODE], name='y-input')
        validate_feed = {x: mnist.validation.images, y_: mnist.validation.labels}
        
        y = inference(x, None)
        
        correct_predict = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
        accuracy = tf.reduce_mean(tf.cast(correct_predict, tf.float32))
        
        variable_averages = tf.train.ExponentialMovingAverage(MOVING_AVERAGE_DECAY)
        variables_to_restore = variable_averages.variables_to_restore()
        saver = tf.train.Saver(variables_to_restore)
        
        while True:
            with tf.Session() as sess:
                ckpt = tf.train.get_checkpoint_state(MODEL_SAVE_PATH)
                if ckpt and ckpt.model_checkpoint_path:
                    saver.restore(sess, ckpt.model_checkpoint_path)
                    global_step = ckpt.model_checkpoint_path.split('/')[-1].split('-')[-1]
                    accuracy_score = sess.run(accuracy, feed_dict=validate_feed)
                    print("After %s training steps, validation accuracy = %g" % (global_step, accuracy_score))
                else:
                    print('No checkpoint file found')
                    return
            time.sleep(EVAL_INTERVAL_SECS)
            
if purpose == 'eval':
    if os.path.exists("/Users"):
        mnist = input_data.read_data_sets("/Users/colinzuo/work/github/personal_study/"
                                          "tensorflow/book_caicloud/mnist/", one_hot=True)
    else:
        mnist = input_data.read_data_sets("D:\\work\\DataAnalysis\\study\\mnist", one_hot=True)
        
    evaluate(mnist)

Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/train-images-idx3-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/train-labels-idx1-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/t10k-images-idx3-ubyte.gz
Extracting /Users/colinzuo/work/github/personal_study/tensorflow/book_caicloud/mnist/t10k-labels-idx1-ubyte.gz
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.c

After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30

INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:tensorflow:Restoring parameters from model/model_55.ckpt-30000
After 30000 training steps, validation accuracy = 0.986
INFO:ten